### Spread Option Pricing in a Copula Affine GARCH(P,Q) Model

This Jupyter Notebook is intended to provide a simple interface for the code related to the article ''An Efficient Unified Approach for Spread Option Pricing in a Copula Market Model''.  The code provides a walkthrough from the setup of a Copula object to the pricing of a spread option using some of the methods explored in the study.

In this initial cell we import and load all of the relevant functions that will be used in what follows. Most of the methods are collected under the Copula class.

In [1]:
# import all of the relevant functions

import numpy as np
from numpy.random import normal, uniform
from scipy.special import ndtri, ndtr
import pandas as pd
from scipy.interpolate import interp1d
from pyvinecopulib import Bicop, BicopFamily
import pyvinecopulib as pcop

# copula specific modules
import innovation_gen as ig

import matplotlib.pyplot as plt


# spread option payoff (prices as input)
def payoff_spreadc(s1, s2, k):
    '''


    Parameters
    ----------
    s1 : TYPE
        DESCRIPTION.
    s2 : TYPE
        DESCRIPTION.
    k : TYPE
        DESCRIPTION.

    Returns
    -------
    p : TYPE
        DESCRIPTION.

    '''
    p = s1 - s2 - k
    p[p < 0] = 0
    return p


# function for log-price
def garch_s(h_1, z, s, r):
    # log-price process

    # h_1 = np.max(h_1, 0)
    return s + r - 0.5 * h_1 + np.sqrt(h_1) * z


# function for the variance
def garch_h(h_0, omega, alpha, beta, gamma, z):
    # init
    o = omega
    a = alpha
    b = beta
    g = gamma

    # volatility process
    # h_0 = np.maximum(h_0, 0)
    h = o + b * h_0 + a * pow(z - g * np.sqrt(h_0), 2)
    return h


def garch_simulation(s0, h0, params, r=0., n_steps=30, n_sim=1000):
    '''


    Parameters
    ----------
    s0 : TYPE
        DESCRIPTION.
    h0 : TYPE
        DESCRIPTION.
    params : TYPE
        DESCRIPTION.
    r : TYPE, optional
        DESCRIPTION. The default is 0..
    n_steps : TYPE, optional
        DESCRIPTION. The default is 30.
    n_sim : TYPE, optional
        DESCRIPTION. The default is 1000.

    Returns
    -------
    TYPE
        DESCRIPTION.

    '''

    # init
    o, a, b, g = params
    s1 = np.log(s0)

    # std normal innovations
    z = normal(0, 1, n_sim)

    h1 = garch_h(h0, o, a, b, g, z)
    # generate s and h
    for i in range(n_steps):
        z = normal(0, 1, n_sim)
        # first asset price process
        s1 = garch_s(h1, z, s1, r)
        h1 = garch_h(h1, o, a, b, g, z)

    return np.exp(s1)


def A_loop(a, b, r_f, phi, omega, alpha):
    a = a + phi * r_f + b * omega - 0.5 * np.log(1 - 2 * alpha * b)
    return a


def B_loop(b, phi, alpha, beta, gam, lam):
    b = phi * (lam + gam) - 0.5 * gam ** 2 + beta * b + (0.5 * (phi - gam) ** 2) / (1 - 2 * alpha * b)
    return b


def pdf_func(s0, h, params, support, steps, r=0., up_lim=100, low_lim=0, prec=10000):
    '''


    Parameters
    ----------
    s0 : TYPE
        DESCRIPTION.
    h : TYPE
        DESCRIPTION.
    params : TYPE
        DESCRIPTION.
    support : TYPE
        DESCRIPTION.
    steps : TYPE
        DESCRIPTION.
    r : TYPE, optional
        DESCRIPTION. The default is 0..
    up_lim : TYPE, optional
        DESCRIPTION. The default is 100.
    low_lim : TYPE, optional
        DESCRIPTION. The default is 0.
    prec : TYPE, optional
        DESCRIPTION. The default is 10000.

    Returns
    -------
    TYPE
        DESCRIPTION.

    '''

    # init
    omega, alpha, beta, gam = params
    lam = -0.5

    s = s0

    hi = float(up_lim - low_lim) / prec
    t = np.linspace(low_lim + hi / 2, up_lim - hi / 2, prec)
    t[t == 0] = 1e-10

    a = np.zeros(len(t))
    b = np.zeros(len(t))

    it = t * complex(0, 1)
    for i in range(steps):
        a = A_loop(a, b, r, it, omega, alpha)
        b = B_loop(b, it, alpha, beta, gam, lam)

    vec = support.reshape(-1, 1)

    f = s ** it
    g = np.exp(a + b * h)
    func = np.real(np.exp(-it * vec) * f * g)

    area = func * hi

    return area.sum(axis=1) / np.pi


def cdf_func(s0, h, params, support, steps, r=0., up_lim=100, low_lim=0, prec=1000):
    '''


    Parameters
    ----------
    s0 : TYPE
        DESCRIPTION.
    h : TYPE
        DESCRIPTION.
    params : TYPE
        DESCRIPTION.
    support : TYPE
        DESCRIPTION.
    steps : TYPE
        DESCRIPTION.
    r : TYPE, optional
        DESCRIPTION. The default is 0..
    up_lim : TYPE, optional
        DESCRIPTION. The default is 100.
    low_lim : TYPE, optional
        DESCRIPTION. The default is 0.
    prec : TYPE, optional
        DESCRIPTION. The default is 1000.

    Returns
    -------
    TYPE
        DESCRIPTION.

    '''

    # init
    omega, alpha, beta, gam = params
    lam = -0.5

    s = s0

    hi = float(up_lim - low_lim) / prec
    t = np.linspace(low_lim + hi / 2, up_lim - hi / 2, prec)
    t[t == 0] = 1e-10

    a = np.zeros(len(t))
    b = np.zeros(len(t))

    it = t * complex(0, 1)
    for i in range(steps):
        a = A_loop(a, b, r, it, omega, alpha)
        b = B_loop(b, it, alpha, beta, gam, lam)

    vec = support.reshape(-1, 1)

    f = s ** it
    g = np.exp(a + b * h)
    j = np.exp(-it * vec) * f * g
    func = np.real(j / it)

    area = func * hi

    return 0.5 - area.sum(axis=1) / np.pi


def pdf_ret(h, params, support, steps, r=0., up_lim=100, low_lim=0, prec=10000):
    '''


    Parameters
    ----------
    s0 : TYPE
        DESCRIPTION.
    h : TYPE
        DESCRIPTION.
    params : TYPE
        DESCRIPTION.
    support : TYPE
        DESCRIPTION.
    steps : TYPE
        DESCRIPTION.
    r : TYPE, optional
        DESCRIPTION. The default is 0..
    up_lim : TYPE, optional
        DESCRIPTION. The default is 100.
    low_lim : TYPE, optional
        DESCRIPTION. The default is 0.
    prec : TYPE, optional
        DESCRIPTION. The default is 10000.

    Returns
    -------
    TYPE
        DESCRIPTION.

    '''

    # init
    omega, alpha, beta, gam = params
    lam = -0.5

    hi = float(up_lim - low_lim) / prec
    t = np.linspace(low_lim + hi / 2, up_lim - hi / 2, prec)
    t[t == 0] = 1e-10

    a = np.zeros(len(t))
    b = np.zeros(len(t))

    it = t * complex(0, 1)
    for i in range(steps):
        a = A_loop(a, b, r, it, omega, alpha)
        b = B_loop(b, it, alpha, beta, gam, lam)

    # FIX THE FUNCTION HERE
    vec = support.reshape(-1, 1)

    g = np.exp(a + b * h)
    func = np.real(np.exp(-it * vec) * g)

    area = func * hi

    return area.sum(axis=1) / np.pi


def cdf_ret(h, params, support, steps, r=0., up_lim=100, low_lim=0, prec=1000):
    '''


    Parameters
    ----------
    s0 : TYPE
        DESCRIPTION.
    h : TYPE
        DESCRIPTION.
    params : TYPE
        DESCRIPTION.
    support : TYPE
        DESCRIPTION.
    steps : TYPE
        DESCRIPTION.
    r : TYPE, optional
        DESCRIPTION. The default is 0..
    up_lim : TYPE, optional
        DESCRIPTION. The default is 100.
    low_lim : TYPE, optional
        DESCRIPTION. The default is 0.
    prec : TYPE, optional
        DESCRIPTION. The default is 1000.

    Returns
    -------
    TYPE
        DESCRIPTION.

    '''

    # init
    omega, alpha, beta, gam = params
    lam = -0.5

    hi = float(up_lim - low_lim) / prec
    t = np.linspace(low_lim + hi / 2, up_lim - hi / 2, prec)
    t[t == 0] = 1e-10

    a = np.zeros(len(t))
    b = np.zeros(len(t))

    it = t * complex(0, 1)
    for i in range(steps):
        a = A_loop(a, b, r, it, omega, alpha)
        b = B_loop(b, it, alpha, beta, gam, lam)

    vec = support.reshape(-1, 1)

    g = np.exp(a + b * h)
    j = np.exp(-it * vec) * g
    func = np.real(j / it)

    area = func * hi
    return 0.5 - area.sum(axis=1) / np.pi

def heston_convert(params):

    # init
    omega, alpha, beta, gam = params
    delta = 1/252
    delta_sq = delta*delta

    lam = 0
    rho = 1 if gam >= 0 else -1
    sigma = np.sqrt(4*alpha/delta_sq)

    k = 2/delta - gam*sigma
    theta = 1/k * (omega/delta_sq + 0.25*sigma*sigma)
    return [k, theta, sigma, rho, lam]


def cdf_ret_heston(h, params, support, steps, r=0., up_lim=100, low_lim=0, prec=1000):

    # init
    k, theta, sigma, rho, lam = params

    u2 = -1/2
    sigma_sq = sigma*sigma
    abar = k*theta # k, theta params
    b2 = k + lam

    tau = steps/252 # time to maturity expressed in years

    hi = float(up_lim - low_lim) / prec
    t = np.linspace(low_lim + hi / 2, up_lim - hi / 2, prec)
    t[t == 0] = 1e-10

    it = t * complex(0, 1)

    # new
    d = np.sqrt((rho*sigma*it - b2)**2 - sigma_sq * (2*u2*it - t**2))
    g = (b2 - rho*sigma*it + d)/(b2 - rho*sigma*it - d)

    # new
    a = r*it*tau + abar/sigma_sq * ((b2 - rho*sigma*it + d)*tau - 2*np.log((1-g*np.exp(d*tau))/(1-g)))
    b = (b2 - rho*sigma*it + d)/sigma_sq * ((1-np.exp(d*tau))/(1-g*np.exp(d*tau)))

    vec = support.reshape(-1, 1)

    gg = np.exp(a + b * h) # h initial value of variance
    j = np.exp(-it * vec) * gg
    func = np.real(j / it)

    area = func * hi
    res = 0.5 - area.sum(axis=1) / np.pi

    # stylized behaviour on the tails
    # idx = np.max(np.where(res <= 0)[0])
    # res[:idx] = 1e-10
    # idx = np.min(np.where(res >= 1)[0])
    # res[idx:] = 1

    return res

def cdf_ret_heston_v2(v, params, support, steps, r=0., up_lim=100, low_lim=0, prec=1000):
    # init
    lambd, vbar, eta, rho, lam = params
    tau = steps  # time to maturity expressed in years

    hi = float(up_lim - low_lim) / prec
    k = np.linspace(low_lim + hi / 2, up_lim - hi / 2, prec)
    k[k == 0] = 0

    it = k * complex(0, 1)

    b = lambd + 1j * rho * eta * k
    d = np.sqrt(b ** 2 + (eta ** 2) * k * (k - 1j))
    g = (b - d) / (b + d)
    T_m = (b - d) / (eta ** 2)
    T = T_m * (1 - np.exp(-d * tau)) / (1 - g * np.exp(-d * tau))
    W = lambd * vbar * (tau * T_m - 2 * np.log((1 - g * np.exp(-d * tau)) / (1 - g)) / (eta ** 2))

    vec = support.reshape(-1, 1)

    gg = np.exp(W + v * T)  # h initial value of variance
    j = np.exp(-it * vec) * gg
    func = np.real(j / it)

    area = func * hi
    return 0.5 - area.sum(axis=1) / np.pi


# δC/δu
def delta_cu(u, v, theta, type=None, copula='plackett'):
    # init
    t = theta
    u[u <= 0] = 0
    v[v <= 0] = 0
    u[u >= 1] = 1
    v[v >= 1] = 1

    if copula == 'plackett':
        tmp = -(t - 1) * (u + v) + 2 * t * v - 1
        rad = (1 + (t - 1) * (u + v)) ** 2 - 4 * u * v * t * (t - 1)
        res = 0.5 * ((tmp / np.sqrt(rad)) + 1)
        return res
    elif copula == 'pyvinecopula':
        u_reshape = u.reshape(-1, 1)
        v_reshape = v.reshape(-1, 1)
        u_tot = np.hstack([u_reshape, v_reshape])
        return type.hfunc1(u_tot)
    elif copula == 'frank':
        up = np.exp(-t * v - t * u) - np.exp(-t * u)
        d = np.exp(-t) - 1 + (np.exp(-t * u) - 1) * (np.exp(-t * v) - 1)
        return up / d
    elif copula == 'amh':
        up = v * (1 - t * (1 - v))
        d = 1 - t * (1 - u) * (1 - v)
        dsquared = d * d
        return up / dsquared
    elif copula == 'fgm':
        return v + t * v * (1 - v) * (1 - 2 * u)
    else:
        print('The selected copula has not been implemented yet.')
        print('Please select between "plackett","frank","amh","fgm".')


#  δC/δv
def delta_cv(u, v, theta, type=None, copula='plackett'):
    return delta_cu(v, u, theta, type, copula)


# general copula class to collect parameters
class Copula:
    '''

    Attributes
    ----------


    Methods
    -------

    '''

    def __init__(self):
        self.s1 = None
        self.s2 = None
        self.var1 = None
        self.var2 = None
        self.g_params1 = None
        self.g_params2 = None
        self.theta = None

    def setup(self, s1, s2, var1, var2, gp1, gp2, theta):
        self.s1 = s1
        self.s2 = s2
        self.var1 = var1
        self.var2 = var2
        self.g_params1 = gp1
        self.g_params2 = gp2
        self.theta = theta
        print('Copula set up.')

    # change the value of theta to a new value
    def set_theta(self, new_theta):
        self.theta = new_theta

    # COPULA SPECIFIC FUNCTIONS
    # ====================================#

    def plackett_innovations(self, u):  # u vector of rvs, t theta correlation parameters
        '''


        Parameters
        ----------
        u : TYPE
            DESCRIPTION.

        Returns
        -------
        v : TYPE
            DESCRIPTION.

        '''

        t = self.theta
        size = u.shape
        z = uniform(0, 1, size=size)  # instrumental uniform rvs
        a = z * (1 - z)

        b = np.sqrt(t) * np.sqrt(t + 4 * a * u * (1 - u) * pow(1 - t, 2))
        v_up = 2 * a * (u * t * t + 1 - u) + t * (1 - 2 * a) - (1 - 2 * z) * b
        v_down = 2 * t + 2 * a * pow(t - 1, 2)
        v = v_up / v_down
        return v

    # function to generate a list of correlated innovations
    def gen_innovations(self, copula, size):  # size = n_steps(rows) * n_sim(columns)
        '''


        Parameters
        ----------
        size : TYPE
            DESCRIPTION.

        Returns
        -------
        z1 : TYPE
            DESCRIPTION.
        TYPE
            DESCRIPTION.

        '''

        # init
        size = (size[0] + 1, size[1])

        z1 = normal(0, 1, size)  # vector of innovations for garch 1
        z1 /= z1.std(axis=1).reshape(-1, 1)  # standardize the generated values for more accurate values
        u = ndtr(z1)  # vector of transformed uniform rvs

        # add new parameter to choose innovation type (for now: 'plackett', 'frank', 'amh', 'fgm')
        if copula == 'plackett':
            v = ig.plackett_innovations(self.theta, u)
        elif copula == 'frank':
            v = ig.frank_innovations(self.theta, u)
        elif copula == 'amh':
            v = ig.amh_innovations(self.theta, u)
        elif copula == 'fgm':
            v = ig.fgm_innovations(self.theta, u)
        else:
            print('The specified copula type has not been implemented!')
            v = ig.plackett_innovations(self.theta, u)  # plackett specific function, can be turned to any copula

        z2 = ndtri(v)
        return z1, z2  # returns a tuple of numpy arrays

    # joint probability distribution under plackett copula
    def joint_dist(self, u, v):
        '''


        Parameters
        ----------
        u : TYPE
            DESCRIPTION.
        v : TYPE
            DESCRIPTION.

        Returns
        -------
        TYPE
            DESCRIPTION.

        '''
        theta = self.theta
        p1 = theta * (1 + (u - 2 * u * v + v) * (theta - 1))
        p2 = (((1 + (theta - 1) * (u + v)) ** 2) - 4 * u * v * theta * (theta - 1)) ** (3 / 2)

        return p1 / p2

    # ====================================#

    # step-wise simulation
    def sim_prices(self, copula, n_steps, r=0., n_sim=1000):
        '''


        Parameters
        ----------
        n_steps : TYPE
            DESCRIPTION.
        r : TYPE, optional
            DESCRIPTION. The default is 0..
        n_sim : TYPE, optional
            DESCRIPTION. The default is 1000.

        Returns
        -------
        TYPE
            DESCRIPTION.

        '''
        # init
        size = (n_steps, n_sim)  # size = n_steps(rows) * n_sim(columns)
        o1, a1, b1, g1 = self.g_params1
        o2, a2, b2, g2 = self.g_params2
        h1 = self.var1
        h2 = self.var2

        s1 = np.log(self.s1)
        s2 = np.log(self.s2)

        z1, z2 = self.gen_innovations(copula, size)

        h1 = garch_h(h1, o1, a1, b1, g1, z1[0])
        h2 = garch_h(h2, o2, a2, b2, g2, z2[0])
        # generate s and h
        for i in range(1, n_steps + 1):
            # first asset price process
            s1 = garch_s(h1, z1[i], s1, r)
            h1 = garch_h(h1, o1, a1, b1, g1, z1[i])
            # second asset price process
            s2 = garch_s(h2, z2[i], s2, r)
            h2 = garch_h(h2, o2, a2, b2, g2, z2[i])

        return np.exp(s1), np.exp(s2)

    # simulation routine using distribution at expiry
    def cdf_simulation(self, n_steps, k, r=0., n_sim=1000, cinterval=False, copula='plackett',
                       object=None):  # cinterval stands for confidence intervals
        '''


        Parameters
        ----------
        n_steps : TYPE
            DESCRIPTION.
        k : TYPE
            DESCRIPTION.
        r : TYPE, optional
            DESCRIPTION. The default is 0..
        n_sim : TYPE, optional
            DESCRIPTION. The default is 1000.
        cinterval : TYPE, optional
            DESCRIPTION. The default is False.

        Returns
        -------
        p_off : TYPE
            DESCRIPTION.

        '''
        t = self.theta
        up_lim = 150  # initial guess, needs to be fine-tuned to increase precision
        # up_lim = 100
        support = np.linspace(0, np.log(up_lim), 5000)
        F2 = cdf_func(self.s2, self.var2, self.g_params2, support, n_steps, r=r)

        # construction of inverse CDF
        tmp = pd.DataFrame(F2, support)
        tmp.drop_duplicates(keep='first', inplace=True)
        tmp.reset_index(inplace=True)
        tmp.columns = ['support', 'F2']

        F2_inv = interp1d(tmp.F2, tmp.support, kind='cubic')

        # simulation of GARCH paths
        st_1 = garch_simulation(self.s1, self.var1, self.g_params1, r=r, n_steps=n_steps, n_sim=n_sim)

        # u = self.cls1.cdf_func(np.log(s1),n_steps)
        u = cdf_func(self.s1, self.var1, self.g_params1, np.log(st_1), steps=n_steps, r=r)

        # add new parameter to choose innovation type (for now: 'plackett', 'frank', 'amh', 'fgm')
        if copula == 'plackett':
            v = ig.plackett_innovations(self.theta, u)
        elif copula == 'pyvinecopula':
            h = uniform(0, 1, u.shape)
            u_tot = np.vstack([u, h]).transpose()
            v = object.hinv1(u_tot)

        #     v = ig.frank_innovations(self.theta, u)
        # elif copula == 'amh':
        #     v = ig.amh_innovations(self.theta, u)
        # elif copula == 'fgm':
        #     v = ig.fgm_innovations(self.theta, u)

        else:
            print('The specified copula type has not been implemented!')
            v = ig.plackett_innovations(self.theta, u)  # plackett specific function, can be turned to any copula

        s2 = np.exp(F2_inv(v))

        k = np.array(k).reshape(-1, 1)
        p_off = st_1 - s2 - k
        p_off[p_off < 0] = 0
        p_off = p_off * np.exp(-r * (n_steps / 252))

        try:
            std = p_off.std(axis=1)
            pc = p_off.mean(axis=1)
        except:
            std = p_off.std()
            pc = p_off.mean()

        instr = 1.96 * (std / np.sqrt(n_sim))
        up_bound = pc + instr
        low_bound = pc - instr

        if cinterval:
            return np.column_stack((k, pc, up_bound, low_bound, std))
        else:
            return pc

    # old analytical option price formula with the double integral
    def analytical_option_price_dint(self, r, k, nsteps, prec=10000):
        '''


        Parameters
        ----------
        r : TYPE
            DESCRIPTION.
        k : TYPE
            DESCRIPTION.
        nsteps : TYPE
            DESCRIPTION.
        prec : TYPE, optional
            DESCRIPTION. The default is 10000.

        Returns
        -------
        TYPE
            DESCRIPTION.

        '''

        # init
        s1, s2 = self.s1, self.s2
        var1, var2 = self.var1, self.var2
        params1, params2 = self.g_params1, self.g_params2
        theta = self.theta
        k = np.array(k)

        k[k == 0] = 1e-10

        # integral support for S2
        y_up = 10
        y_low = -10
        h_y = float(y_up - y_low) / prec
        y = np.linspace(y_low + h_y / 2, y_up - h_y / 2, prec)
        y[y == 0] = 1e-10
        y = y.reshape(-1, 1)

        # integral support for S1
        x_up = 10
        x_low = -10
        h_x = float(x_up - x_low) / prec
        x = np.linspace(x_low + h_x / 2, x_up - h_x / 2, prec)
        x[x == 0] = 1e-10

        multi_h = h_x * h_y

        # x,y are the variables to integrate for
        f1 = pdf_func(s1, var1, params1, x, steps=nsteps)  # standard precision is 10000
        f2 = pdf_func(s2, var2, params2, y, steps=nsteps)

        F1 = cdf_func(s1, var1, params1, x, steps=nsteps)
        F2 = cdf_func(s2, var2, params2, y, steps=nsteps)

        # matrix of copula results
        F2 = F2.reshape(-1, 1)
        c_matrix = self.joint_dist(F1, F2)  # COPULA SPECIFIC FUNCTION TO SETUP

        # matrix of combinations of products of f(x)*f(y) for all x,y
        f2 = f2.reshape(-1, 1)
        pdf_matrix = f1 * f2

        # converting log-prices back to prices
        x = np.exp(x)
        y = np.exp(y)

        # payoff matrix
        res_list = []
        try:
            for strike in k:
                strike = strike.reshape(-1, 1)
                poff_matrix = payoff_spreadc(x, y, strike)
                final_m = c_matrix * pdf_matrix * poff_matrix
                # application of the midpoint rule
                final_m = final_m * multi_h
                res_list.append(np.exp(-r * (nsteps / 252)) * np.sum(final_m))
            return res_list

        except:
            poff_matrix = payoff_spreadc(x, y, k)

            final_m = c_matrix * pdf_matrix * poff_matrix
            # application of the midpoint rule
            final_m = final_m * multi_h

            res = np.exp(-r * (nsteps / 252)) * np.sum(final_m)
            return res

    # new single integral option price
    def analytical_price(self, r, k, nsteps, int_style='midpoint', copula='plackett', char='HN',prec=10000):

        # init
        s1, s2 = self.s1, self.s2
        var1, var2 = self.var1, self.var2
        params1, params2 = self.g_params1, self.g_params2
        theta = self.theta
        k = np.array(k)

        k[k == 0] = 1e-10
        k_reshape = k.reshape(-1, 1)

        def d1(s1, s2, k, v_vec):

            k_reshape = k.reshape(-1, 1)
            tmp = (s2 * np.exp(F2_inv(v_vec)) + k_reshape) / s1
            ppts = np.log(tmp)

            return F1_interp(ppts)

        def d2(s1, s2, k, u_vec):

            tmp = (s1 * np.exp(F1_inv(u_vec)) - k) / s2
            ppts = np.log(tmp)
            return F2_interp(ppts)

        support = np.linspace(-2, 5, 1000)

        if char == 'HN':
            # calibration of F1 inverse + F1 interpolation
            F1 = cdf_ret(var1, params1, support, steps=nsteps)
        elif char == 'Heston':
            # calibration of F1 inverse + F1 interpolation with Heston model
            F1 = cdf_ret_heston(var1, params1, support, steps=nsteps)  # REMEMBER CHANGE PARAMS
        elif char == "VG":
            pass


        tmp = pd.DataFrame(F1, support)
        tmp.drop_duplicates(keep='first', inplace=True)
        tmp.reset_index(inplace=True)
        tmp.columns = ['support', 'F1']

        F1_inv = interp1d(tmp.F1, tmp.support, kind='cubic', fill_value='extrapolate')
        F1_interp = interp1d(tmp.support, tmp.F1, kind='cubic', fill_value='extrapolate')

        if char == 'HN':
            # calibration of F1 inverse + F1 interpolation
            F2 = cdf_ret(var2, params2, support, steps=nsteps)
        elif char == 'Heston':
            # calibration of F1 inverse + F1 interpolation with Heston model
            F2 = cdf_ret_heston(var2, params2, support, steps=nsteps)  # REMEMBER CHANGE PARAMS
        elif char == "VG":
            pass

        tmp = pd.DataFrame(F2, support)
        tmp.drop_duplicates(keep='first', inplace=True)
        tmp.reset_index(inplace=True)
        tmp.columns = ['support', 'F2']

        F2_inv = interp1d(tmp.F2, tmp.support, kind='linear', fill_value='extrapolate')
        F2_interp = interp1d(tmp.support, tmp.F2, kind='linear', fill_value='extrapolate')

        # numerical integration methods
        def integrate_midpoint(k, k_reshape, prec, s1, s2, theta, copula):

            # INTEGRAL 1
            d3 = np.full(len(np.atleast_1d(k)), 1e-20)
            mask = k > 0
            d3[mask] = F1_interp(np.log(k[mask] / s1))
            d3[d3 <= 1e-20] = 1e-20

            hi = 1. / prec
            t = np.linspace(d3 + hi / 2, 1 - hi / 2, prec)

            p1 = s1 * np.exp(F1_inv(t))

            if copula == 'plackett':
                p2 = delta_cu(t, d2(s1, s2, k, t), theta, copula=copula)
            else:
                p2 = delta_cu(t, d2(s1, s2, k, t), theta, copula='pyvinecopula', type=copula)
                p2 = p2.reshape(-1, 1)

            func = p1 * p2 * hi
            integral_1 = np.sum(func, axis=0)

            # INTEGRAL 2
            t = np.linspace(0 + hi / 2, 1 - hi / 2, prec)

            # delta C/ delta v

            if copula == 'plackett':
                p1 = delta_cv(d1(s1, s2, k, t), t, theta, copula=copula)
            else:
                p1 = delta_cv(d1(s1, s2, k, t), t, theta, copula='pyvinecopula', type=copula)

            p2 = s2 * np.exp(F2_inv(t)) + k_reshape

            # TO DO
            # handle risk free rate

            p3 = s2 + k

            func = p1 * p2 * hi

            integral_2 = p3 - np.sum(func, axis=1)

            return integral_1 - integral_2

        def integrate_simpson(k, k_reshape, prec, s1, s2, theta, copula):

            # INTEGRAL 1
            # d3 = np.full(len(np.atleast_1d(k)), 1e-20)
            # mask = k > 0
            # d3[mask] = F1_interp(np.log(k[mask]/ s1))
            # d3[d3 <= 1e-20] = 1e-20

            if prec % 2 == 0:
                prec += 1

            # streamline this
            d3 = np.full(len(np.atleast_1d(k)), 0.)
            mask = k > 0
            d3[mask] = F1_interp(np.log(k[mask] / s1))
            d3[d3 <= 0.] = 0.

            hi = 1. / prec
            t = np.linspace(d3, 1, prec)

            n = prec

            p1 = s1 * np.exp(F1_inv(t))

            if copula == 'plackett':
                p2 = delta_cu(t, d2(s1, s2, k, t), theta, copula=copula)
            else:
                p2 = delta_cu(t, d2(s1, s2, k, t), theta, copula='pyvinecopula', type=copula)
                p2 = p2.reshape(-1, 1)

            func = p1 * p2

            # here is the simpson rule method part
            integral_1_simp = (hi / 3) * (
                        func[0] + 2 * np.sum(func[2:n - 2:2], axis=0) + 4 * np.sum(func[1:n - 1:2], axis=0) + func[
                    n - 1])
            # integral_1_simp = hi / 3 * np.sum(func[0:-1:2] + 4 * func[1::2] + func[2::2])

            # INTEGRAL 2
            t = np.linspace(0, 1, prec)

            # def d1(s1, s2, k, v_vec):
            #     tmp = (s2 * np.exp(F2_inv(v_vec)) + k) / s1
            #     ppts = np.log(tmp)
            #
            #     return F1_interp(ppts)

            # delta C/ delta v
            if copula == 'plackett':
                p1 = delta_cv(d1(s1, s2, k, t), t, theta, copula=copula)
            else:
                p1 = delta_cv(d1(s1, s2, k, t), t, theta, copula='pyvinecopula', type=copula)

            p2 = s2 * np.exp(F2_inv(t)) + k_reshape

            # TO DO

            # handle risk free rate
            p3 = s2 + k

            func = p1 * p2

            simp_2 = (hi / 3) * (func[:, 0] + 2 * np.sum(func[:, 2:n - 2:2], axis=1) + 4 * np.sum(func[:, 1:n - 1:2],
                                                                                                  axis=1) + func[:,
                                                                                                            n - 1])
            # simp_2 = hi/3 * np.sum(func[0:-1:2] + 4*func[1::2] + func[2::2])
            integral_2_simp = p3 - simp_2
            return integral_1_simp - integral_2_simp

        if int_style == 'midpoint':
            res_mid = integrate_midpoint(k, k_reshape, prec, s1, s2, theta, copula)
            return res_mid

        elif int_style == 'simpson':
            res_simp = integrate_simpson(k, k_reshape, prec, s1, s2, theta, copula)
            return res_simp
        else:
            print('The selected integration method is not implmented.')
            print('Please select between "midpoint" or "simpson".')

    # step-wise option prices
    def simulation_option_price(self, copula, n_steps, k, r=0., n_sim=1000):
        '''


        Parameters
        ----------
        n_steps : TYPE
            DESCRIPTION.
        k : TYPE
            DESCRIPTION.
        r : TYPE, optional
            DESCRIPTION. The default is 0..
        n_sim : TYPE, optional
            DESCRIPTION. The default is 1000.

        Returns
        -------
        TYPE
            DESCRIPTION.

        '''
        s1, s2 = self.sim_prices(copula, n_steps, r, n_sim)
        k = np.array(k).reshape(-1, 1)
        p = payoff_spreadc(s1, s2, k)
        return np.mean(p, axis=1)


In the following cell we set up the parameters of the  two Heston-Nandi GARCH.
$h_i$  with $i =1,2$ are the initial variances.
$s_i$  with $i =1,2$ are the initial stock prices.

$p_1$ and $p_2$ are the vectors containing the parameters of the GARCH in this order:
$$
p_i = \left(\omega_i, \alpha_i, \beta_i, \gamma_i \right) \text{ with } i=1,2.
$$

In [2]:
# specification of the parameters

# variances
h1 = 0.0006014936149641224
h2 = 0.0003988578505117192

# initial prices
s1 = 50.52
s2 = 44.76

# GARCH parameters
p1 = [9.124459397935986e-33, 7.08103198288088e-06, 0.9138527559346877, 96.58684085255285]
p2 = [0.0002845211058232067, 7.155007620662264e-06, 0.17506894272852375, 0.13902479264341316]

In [3]:
# initialization of Copula class
copula = Copula()
copula.setup(s1, s2, h1, h2, p1, p2, 51.2) # 51.2 baseline theta parameter of Plackett Copula

Copula set up.


### Analytical Pricing

We produce a call option price for strike $k$, days to maturity $t$ and number of gridpoints for integration (_i.e._ precision of numerical integration) $n$. We use the Plackett Copula to model the dependence of the underlying processes. Numerical integration method is set to ``midpoint``.

In [4]:
k = 5 # strike price of the option
n = 5000 # gridpoints of the integral
t = 90 # days to maturity
prc = copula.analytical_price(0., k, t, int_style='midpoint',prec=n)[0]

print('The strike price is {}.'.format(k))
print('The number of gridpoints for integration is {}.'.format(n))
print('The number of days to matuirty is {}.'.format(t))
print('The price of the call option computed using the novel analytical formula is: {}'.format(prc))

The strike price is 5.
The number of gridpoints for integration is 5000.
The number of days to matuirty is 90.
The price of the call option computed using the novel analytical formula is: 2.2657809995720584


In the following cell we make use of a loop to price multiple call options with different strikes and different numerical integration precision.

The strike list we use is ``[0, 2.5, 5, 7.5, 10]``. The precision varies in the following list ``[100, 500, 1000, 5000, 10000, 50000]``.

In [5]:
strike_list = np.arange(0., 10.5, 2.5) # produces a list of strike prices from 0 to 10, equally spaced.

# Computation of Call option price
plackett_mid = pd.DataFrame()

for n in [100, 500, 1000, 5000, 10000, 50000]:
    price_vec = []
    time_vec = []
    for k in strike_list:
        price_vec.append(copula.analytical_price(0., k, 90, int_style='midpoint',prec=n)[0])
    plackett_mid['N_' + str(n)] = price_vec

plackett_mid['strike'] = strike_list.tolist()

In the cell below is the table with the price results. Each column contains the price computed at different strikes. Each row contains the prices computed at the same strike but using different degrees of precision of integration (increasing precision from left to right).

In [6]:
plackett_mid[['strike', 'N_100', 'N_500', 'N_1000', 'N_5000', 'N_10000', 'N_50000']]

,strike,N_100,N_500,N_1000,N_5000,N_10000,N_50000
0,0.0,6.081576,6.142016,6.146542,6.148887,6.149118,6.149282
1,2.5,3.963987,4.013559,4.016554,4.018325,4.018532,4.018682
2,5.0,2.223382,2.262014,2.264318,2.265781,2.265958,2.266091
3,7.5,1.072548,1.106827,1.109114,1.110597,1.110757,1.110878
4,10.0,0.492453,0.527960,0.530776,0.532523,0.532702,0.532826


Notice that the same loop can be used to price options using the Simpson integration method. See below.

In [7]:
strike_list = np.arange(0., 10.5, 2.5) # produces a list of strike prices from 0 to 10, equally spaced.

# Computation of Call option price
plackett_simpson = pd.DataFrame()

for n in [100, 500, 1000, 5000, 10000, 50000]:
    price_vec = []
    time_vec = []
    for k in strike_list:
        price_vec.append(copula.analytical_price(0., k, 90, int_style='simpson',prec=n)[0])
    plackett_simpson['N_' + str(n)] = price_vec

plackett_simpson['strike'] = strike_list.tolist()
plackett_simpson[['strike', 'N_100', 'N_500', 'N_1000', 'N_5000', 'N_10000', 'N_50000']]

,strike,N_100,N_500,N_1000,N_5000,N_10000,N_50000
0,0.0,6.249113,6.153383,6.149878,6.148879,6.148955,6.149187
1,2.5,4.073582,4.020054,4.018278,4.018112,4.018278,4.018573
2,5.0,2.283569,2.263088,2.263623,2.265158,2.265520,2.265951
3,7.5,1.097470,1.101732,1.105271,1.109329,1.110015,1.110682
4,10.0,0.493813,0.516736,0.523535,0.530494,0.531570,0.532553


### Montecarlo Pricing

In the following cell we price the option using montecarlo simulation. Please note that, naturally, the simulation is very sensitive to the seed one sets for pseudo-random number generation. We will later provide some examples.

In [8]:
k = 0 # strike price of the option
n = 100000 # number of simulations
t = 90 # days to maturity
mc_prc = copula.cdf_simulation(t, k, n_sim=n, copula='plackett',cinterval=True)[0]

print('The strike price is {}.'.format(k))
print('The number of simulations is {}.'.format(n))
print('The number of days to matuirty is {}'.format(t))
print('The price of the call option computed using montecarlo simulation is: {}'.format(mc_prc))

The strike price is 0.
The number of simulations is 100000.
The number of days to matuirty is 90
The price of the call option computed using montecarlo simulation is: [0.         6.17301858 6.20012873 6.14590843 4.37397069]


The following loop is designed to price several options on different strikes.

In [9]:
# list of strikes
strike_list = np.arange(0., 10.5, 2.5)

price_vec = []
upb_vec = []
lb_vec = []

for k in strike_list:

    # seeds 0, 3, 1
    np.random.seed(3)
    montecarlo = copula.cdf_simulation(90, k, n_sim=100000, copula='plackett',cinterval=True)
    lb_vec.append(montecarlo[0,3])
    upb_vec.append(montecarlo[0,2])
    price_vec.append(montecarlo[0,1])

plackett_montecarlo = pd.DataFrame()
plackett_montecarlo['strike'] = strike_list
plackett_montecarlo['price'] = price_vec
plackett_montecarlo['lower bound'] = lb_vec
plackett_montecarlo['upper bound'] = upb_vec


In [10]:
plackett_montecarlo

,strike,price,lower bound,upper bound
0,0.0,6.173270,6.146136,6.200405
1,2.5,4.034399,4.009927,4.058871
2,5.0,2.272071,2.251379,2.292764
3,7.5,1.111476,1.095208,1.127745
4,10.0,0.533776,0.521508,0.546044


In this cell, the loop is designed to simulate the call option price initializing the random number generator using different seeds. This exercise is done in order to show that the simulation has an inherent variability in terms of simulated price, whilst in general the confidence intervals stay similar to each other.

In [11]:
k = 7.5 # single strike price

price_vec = []
upb_vec = []
lb_vec = []

seed_list = [0, 3, 6, 9, 12]

for i in range(len(seed_list)):
    np.random.seed(seed_list[i])
    montecarlo = copula.cdf_simulation(90, k, n_sim=100000, copula='plackett',cinterval=True)
    lb_vec.append(montecarlo[0,3])
    upb_vec.append(montecarlo[0,2])
    price_vec.append(montecarlo[0,1])

plackett_montecarlo = pd.DataFrame()
plackett_montecarlo['seed'] = seed_list
plackett_montecarlo['price'] = price_vec
plackett_montecarlo['lower bound'] = lb_vec
plackett_montecarlo['upper bound'] = upb_vec

In [12]:
plackett_montecarlo

,seed,price,lower bound,upper bound
0,0,1.112792,1.096501,1.129084
1,3,1.111476,1.095208,1.127745
2,6,1.105975,1.089689,1.122260
3,9,1.100083,1.083924,1.116241
4,12,1.122221,1.105951,1.138491


### Pricing in the Clayton Copula Framework
------
Now we do a pricing exercise using Clayton Copula to capture the relationship between the underlying assets. In our framework, Clayton Copula is the one that takes more to converge to analytical price.

In [13]:
k = 5 # strike price of the option
n = 5000 # gridpoints of the integral
t = 90 # days to maturity

# set up the Claytion Copula with parameter 6.57
clayton = Bicop(family=BicopFamily.clayton, parameters=np.array([6.57]))

prc = copula.analytical_price(0., k, 90, int_style='midpoint', copula=clayton, prec=n)[0]

print('The strike price is {}.'.format(k))
print('The number of gridpoints for integration is {}.'.format(n))
print('The number of days to matuirty is {}.'.format(t))
print('The price of the call option computed using the novel analytical formula is: {}'.format(prc))

The strike price is 5.
The number of gridpoints for integration is 5000.
The number of days to matuirty is 90.
The price of the call option computed using the novel analytical formula is: 2.185507732131299


In [14]:
strike_list = np.arange(0., 10.5, 2.5) # produces a list of strike prices from 0 to 10, equally spaced.

# Computation of Call option price
clayton_mid = pd.DataFrame()

for n in [100, 500, 1000, 5000, 10000, 50000]:
    price_vec = []
    time_vec = []
    for k in strike_list:
        price_vec.append(copula.analytical_price(0., k, 90, int_style='midpoint', copula=clayton, prec=n)[0])
    clayton_mid['N_' + str(n)] = price_vec

clayton_mid['strike'] = strike_list.tolist()

In [15]:
clayton_mid[['strike', 'N_100', 'N_500', 'N_1000', 'N_5000', 'N_10000', 'N_50000']]

,strike,N_100,N_500,N_1000,N_5000,N_10000,N_50000
0,0.0,5.945319,5.975415,5.977702,5.979339,5.979534,5.979681
1,2.5,3.794959,3.819477,3.821630,3.823219,3.823415,3.823561
2,5.0,2.160302,2.181820,2.183934,2.185508,2.185701,2.185849
3,7.5,1.160331,1.181093,1.183180,1.184774,1.184967,1.185115
4,10.0,0.613264,0.634827,0.636990,0.638609,0.638805,0.638954


In [16]:
# list of strikes
strike_list = np.arange(0., 10.5, 2.5)

price_vec = []
upb_vec = []
lb_vec = []

for k in strike_list:

    # seeds 0, 3, 1
    np.random.seed(3)
    montecarlo = copula.cdf_simulation(90, k, n_sim=100000, copula='pyvinecopula', object=clayton ,cinterval=True)
    lb_vec.append(montecarlo[0,3])
    upb_vec.append(montecarlo[0,2])
    price_vec.append(montecarlo[0,1])

clayton_montecarlo = pd.DataFrame()
clayton_montecarlo['strike'] = strike_list
clayton_montecarlo['price'] = price_vec
clayton_montecarlo['lower bound'] = lb_vec
clayton_montecarlo['upper bound'] = upb_vec

In [17]:
clayton_montecarlo

,strike,price,lower bound,upper bound
0,0.0,5.994126,5.965805,6.022447
1,2.5,3.833511,3.807439,3.859582
2,5.0,2.188022,2.165831,2.210214
3,7.5,1.183278,1.165724,1.200831
4,10.0,0.637105,0.623804,0.650406


In the following cells we propose, as above, a comparison of call prices obtained through montecarlo simulation initializing the random number generator with various seeds. 

In [18]:
k = 2.5 # single strike price

price_vec = []
upb_vec = []
lb_vec = []

seed_list = [5, 10, 15, 20]

for i in range(len(seed_list)):
    np.random.seed(seed_list[i])
    montecarlo = copula.cdf_simulation(90, k, n_sim=500000, copula='pyvinecopula', object=clayton ,cinterval=True)
    lb_vec.append(montecarlo[0,3])
    upb_vec.append(montecarlo[0,2])
    price_vec.append(montecarlo[0,1])

clayton_montecarlo = pd.DataFrame()
clayton_montecarlo['seed'] = seed_list
clayton_montecarlo['price'] = price_vec
clayton_montecarlo['lower bound'] = lb_vec
clayton_montecarlo['upper bound'] = upb_vec

In [19]:
clayton_montecarlo

,seed,price,lower bound,upper bound
0,5,3.814036,3.802395,3.825678
1,10,3.834808,3.823084,3.846532
2,15,3.815810,3.804156,3.827465
3,20,3.825458,3.813778,3.837137


We notice that, depending on the seed used for initializing the pseudo-random sequence generation, the Montecarlo price is greater or lower with respect to the price computed using the analytical formula we developed. The analytical price is, however, always within the confindence interval obtained thorugh the Montecarlo Simulation.

This notebook does not intend to be a comprehensive replication of the analysis in the paper. It is only a user-friendly introduction to some of the tools employed in the study. Please contact the authors (edoardo.berton@unicatt.it or lorenzo.mercuri@unimi.it) for any issue, comment or suggestion.